---
title: 'A Word on Performance'
toc: true
number-sections: true
mainfont: Arial
format:
  html: default
jupyter: python3
---

In [1]:
#| echo: false
#| output: false
%load_ext autoreload
%autoreload 2
import sys

sys.path.append(r"..\..\src")

In [2]:
# If you use IPython/Jupyter:
import sys

from powfacpy.base.active_project import ActiveProject

sys.path.append(
    r"C:\Program Files\DIgSILENT\PowerFactory 2025 SP3\Python\3.13"
)  # you may use a different directory

# Get the PF app
import powerfactory
import powfacpy
from powfacpy.pf_class_protocols import PFApp

app: PFApp = powerfactory.GetApplication()
app.Show()
app.ActivateProject(
    r"powfacpy\39_bus_new_england_copy_where_tests_run"
)  # You may change the project path.
act_prj = ActiveProject(app)

In [3]:
import platform

uname = platform.uname()
print(f"System: {uname.system}")
print(f"Release: {uname.release}")
print(f"Version: {uname.version}")
print(f"Machine: {uname.machine}")
print(f"Processor: {uname.processor}")

System: Windows
Release: 11
Version: 10.0.26200
Machine: AMD64
Processor: Intel64 Family 6 Model 140 Stepping 1, GenuineIntel


# Accessing Objects

We will compare the performance of the `get_obj` method of *powfacpy* with the native `GetContents` method of *PowerFactory*. `get_obj` calls `GetContents` under the hood and adds further functionality, checks input arguments and improves exception handling. Hence, we expect that there is some overhead when using `get_obj`. 

The methods are tested while the *PowerFactory* app is shown or hidden as we use *PowerFactory* in *non-interactive (engine)* mode. We use a "try statement" (exception handling construct) to ensure the app will finally be shown even if an exception is thrown during excecution which is good practice.

In [4]:
act_prj_obj = app.GetActiveProject()

print("{:<50}".format("Using `get_obj`:"), end="")
%timeit act_prj.get_obj(r"Network Model\Network Data\Grid\*.ElmTerm")
print("{:<50}".format("Using `GetContents`:"), end="")
%timeit act_prj_obj.GetContents(r"Network Model\Network Data\Grid\*.ElmTerm")

try:
    app.Hide()

    print("{:<50}".format("Using `get_obj` (app is hidden):"), end="")
    %timeit act_prj.get_obj(r"Network Model\Network Data\Grid\*.ElmTerm")
    print("{:<50}".format("Using `GetContents` (app is hidden):"), end="")
    %timeit act_prj_obj.GetContents(r"Network Model\Network Data\Grid\*.ElmTerm")
finally:
    app.Show()

Using `get_obj`:                                  87.6 ms ± 11.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Using `GetContents`:                              45.6 ms ± 9.49 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Using `get_obj` (app is hidden):                  1.05 ms ± 35.9 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
Using `GetContents` (app is hidden):              578 μs ± 18 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


It becomes apparent that `get_obj` is roughly 50% slower. However, the most significant difference in performance originates from the status of the *PowerFactory* app (hidden or not).

Next we compare `get_calc_relevant_obj` of *powfacpy* with `GetCalcRelevantObject`. 

In [5]:
print("{:<50}".format("Using `get_calc_relevant_obj`:"), end="")
%timeit act_prj.get_calc_relevant_obj(r"ElmTerm")
print("{:<50}".format("Using `GetCalcRelevantObject`:"), end="")
%timeit app.GetCalcRelevantObjects(r"ElmTerm")

try:
    app.Hide()

    print("{:<50}".format("Using `get_calc_relevant_obj` (app is hidden):"), end="")
    %timeit act_prj.get_calc_relevant_obj(r"ElmTerm")
    print("{:<50}".format("Using `GetCalcRelevantObject` (app is hidden):"), end="")
    %timeit app.GetCalcRelevantObjects(r"ElmTerm")
finally:
    app.Show()

Using `get_calc_relevant_obj`:                    45.3 ms ± 2.87 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Using `GetCalcRelevantObject`:                    49.3 ms ± 5.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Using `get_calc_relevant_obj` (app is hidden):    1.18 ms ± 368 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
Using `GetCalcRelevantObject` (app is hidden):    1.01 ms ± 89.6 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


The difference in performance between the methods is small, whereas the influence of the status of the app is again very significant. 

# Getting Attributes

In the following it is compared how the reading of attributes using powfacpy's `get_attr` compares to the native `GetAttribute` method. `get_attr` is used in two different ways, either the object (terminal "Bus 01") is provided or its path.

In [6]:
terminal = act_prj.get_unique_obj(r"Network Model\Network Data\Grid\Bus 01")

print("{:<50}".format("Using `get_attr`:"), end="")
%timeit act_prj.get_attr(terminal, "uknom")
print("{:<50}".format("Using `get_attr` and path:"), end="")
%timeit act_prj.get_attr(r"Network Model\Network Data\Grid\Bus 01", "uknom")
print("{:<50}".format("Using `GetAttribute`:"), end="")
%timeit terminal.GetAttribute("uknom")

try:
    app.Hide()

    print("{:<50}".format("Using `get_attr` (app is hidden):"), end="")
    %timeit act_prj.get_attr(terminal, "uknom")
    print("{:<50}".format("Using `get_attr` and path (app is hidden):"), end="")
    %timeit act_prj.get_attr(r"Network Model\Network Data\Grid\Bus 01", "uknom")
    print("{:<50}".format("Using `GetAttribute` (app is hidden):"), end="")
    %timeit terminal.GetAttribute("uknom")
finally:
    app.Show()

Using `get_attr`:                                 4.51 μs ± 553 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
Using `get_attr` and path:                        107 ms ± 11.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Using `GetAttribute`:                             3.46 μs ± 231 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
Using `get_attr` (app is hidden):                 2.75 μs ± 362 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
Using `get_attr` and path (app is hidden):        1.22 ms ± 27 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
Using `GetAttribute` (app is hidden):             2.68 μs ± 199 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


`get_attr` performs similar to `GetAttribute` except when there is an overhead to get the object from its path. It is interesting that influence of the status of the app is minor when reading attributes only.

# Setting Attributes

Now let's check the performance for writing using `set_attr` or `SetAttribute`. Besides hiding the app, we also investigate the cache of the *PowerFactory* interface by executing the `SetWriteCacheEnabled` method (see scripting reference for more information). 

In [7]:
print("{:<50}".format("Using `set_attr`:"), end="")
%timeit act_prj.set_attr(terminal, {"vmax": 1.06, "vmin": 0.01})
print("{:<50}".format("Using `set_attr` and path: "), end="")
%timeit act_prj.set_attr(r"Network Model\Network Data\Grid\Bus 01", {"vmax": 1.06, "vmin": 0.01})
print("{:<50}".format("Using `SetAttribute`:"), end="")
%timeit terminal.SetAttribute("vmax", 1.06), terminal.SetAttribute("vmin", 0.01)

try:
    app.Hide()

    print("{:<50}".format("\nUsing `set_attr` (app is hidden):"), end="")
    %timeit act_prj.set_attr(terminal, {"vmax": 1.06, "vmin": 0.01})
    print("{:<50}".format("Using `set_attr` and path (app is hidden):"), end="")
    %timeit act_prj.set_attr(r"Network Model\Network Data\Grid\Bus 01", {"vmax": 1.06, "vmin": 0.01})
    print("{:<50}".format("Using `SetAttribute` (app is hidden):"), end="")
    %timeit terminal.SetAttribute("vmax", 1.06), terminal.SetAttribute("vmin", 0.01)
    
    print("{:<50}".format("\nExecute `SetWriteCacheEnabled`:"), end="")
    %timeit app.SetWriteCacheEnabled(1)
    print("{:<50}".format("Using `set_attr` (app hidden, cache enabled):"), end="")
    %timeit act_prj.set_attr(terminal, {"vmax": 1.06, "vmin": 0.01})
    print("{:<50}".format("Using `set_attr` and path (app hidden, cache enabled):"), end="")
    %timeit act_prj.set_attr(r"Network Model\Network Data\Grid\Bus 01", {"vmax": 1.06, "vmin": 0.01})
    print("{:<50}".format("Using `SetAttribute` (app hidden, cache enabled):"), end="")
    %timeit terminal.SetAttribute("vmax", 1.06), terminal.SetAttribute("vmin", 0.01)

    print("{:<50}".format("Execute `WriteChangesToDb`:"), end="")
    %timeit app.WriteChangesToDb()
    app.SetWriteCacheEnabled(0)
    
finally:
    app.Show()

Using `set_attr`:                                 193 ms ± 25.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Using `set_attr` and path:                        309 ms ± 25.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Using `SetAttribute`:                             191 ms ± 6.74 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

Using `set_attr` (app is hidden):                7.36 ms ± 954 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Using `set_attr` and path (app is hidden):        9.57 ms ± 896 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Using `SetAttribute` (app is hidden):             6.32 ms ± 471 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Execute `SetWriteCacheEnabled`:                  309 ns ± 29.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
Using `set_attr` (app hidden, cache enabled):     2.7 ms ± 325 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Using `set_attr` and path (app hid

Overall writing is the most computationally expensive action. The impact of the method used is minor whereas the status of the app is again crucial. Using `SetWriteCacheEnabled` reduces the execution time by roughly 50%. However, there is some overhead due to the execution of `SetWriteCacheEnabled` and `WriteChangesToDb` this approach is recommended for larger chunks of data to be written to the *PowerFactory* database. 

# Caching

Caching is the technique of storing previously computed or frequently accessed data in a temporary storage layer so that subsequent requests can be served faster, reducing latency, computational cost, or load on underlying systems.

*powfacpy* offers a few features to improve the performance based on caching data. For exmaple, the return value of a method can be cached so that later method calls become much faster. Note however that this is only applicable to methods without input arguments and the retun value (the name of the class of the object in the following example) must not change.

In [8]:
from powfacpy.applications.caching import cache_method

terminal = act_prj.get_unique_obj(r"Network Model\Network Data\Grid\Bus 01")
print("{:<50}".format("Using `GetClassName`:"), end="")
%timeit terminal.GetClassName()
cache_method(terminal, "GetClassName")
print("{:<50}".format("Using `GetClassName` after caching:"), end="")
%timeit terminal.GetClassName()

try:
    app.Hide()

    terminal = act_prj.get_unique_obj(r"Network Model\Network Data\Grid\Bus 01")
    print("{:<50}".format("Using `GetClassName` (app is hidden):"), end="")
    %timeit terminal.GetClassName()
    cache_method(terminal, "GetClassName")
    print("{:<50}".format("Using `GetClassName` after caching (app is hidden):"), end="")
    %timeit terminal.GetClassName()
finally:
    app.Show()


Using `GetClassName`:                             47 ms ± 3.45 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Using `GetClassName` after caching:               122 ns ± 17.1 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
Using `GetClassName` (app is hidden):             592 μs ± 72 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
Using `GetClassName` after caching (app is hidden):63.9 ns ± 1.34 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


The performance improves tremendously. This way of caching the return value of a method is a more readable alternative to assigning the value to a variable and pass that variable to different places in your code.

Another simple way of improving the perfomance is to use the `ActiveProjectCached` class as an alternative to `ActiveProject`. This class caches properties such as the `study_cases_folder` (under the hood the `@cached_property` decorator is used). Note that this class can be used only for one *PowerFactory* project that remains active and not for different projects.  

In [9]:
from powfacpy.base.active_project import ActiveProjectCached

act_prj_cached = ActiveProjectCached(app)

print("{:<50}".format("Using `study_cases_folder`:"), end="")
%timeit act_prj.study_cases_folder
print("{:<50}".format("Using `study_cases_folder` with cached properties:"), end="")
%timeit act_prj_cached.study_cases_folder

try:
    app.Hide()

    print("{:<50}".format("Using `study_cases_folder` (app is hidden):"), end="")
    %timeit act_prj.study_cases_folder
    print("{:<50}".format("Using `study_cases_folder` with cached properties (app is hidden):"), end="")
    %timeit act_prj_cached.study_cases_folder
finally:
    app.Show()


Using `study_cases_folder`:                       51.9 ms ± 4.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Using `study_cases_folder` with cached properties:41.2 ns ± 2.33 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
Using `study_cases_folder` (app is hidden):       590 μs ± 38.6 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
Using `study_cases_folder` with cached properties (app is hidden):29.7 ns ± 2.09 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
